In [6]:
import torch
from avalanche.training.supervised import ICaRL
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import TensorboardLogger
from torch.utils.tensorboard import SummaryWriter
from avalanche.evaluation.metrics import accuracy_metrics, loss_metrics, timing_metrics, forgetting_metrics,class_accuracy_metrics
from avalanche.benchmarks.classic import CORe50
from avalanche.logging import InteractiveLogger
from SimpleCNN_icarl import *
from avalanche.benchmarks.utils import as_avalanche_dataset

In [7]:
from torchvision import transforms

# Trasformazione per ridurre la risoluzione delle immagini
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Riduci la risoluzione a 64x64 (o altra risoluzione desiderata)
    transforms.ToTensor()
])

In [8]:
# Caricamento del dataset CORe50
benchmark = CORe50(scenario="nc",run=0,object_lvl=False,train_transform=transform,eval_transform=transform)
log_dir = './tb_data/experiment_SimpleCNN_ICarl'

# Creazione del logger di TensorBoard per run1
tb_logger_run1 = TensorboardLogger(tb_log_dir=log_dir)

# Lista dei modelli
print("Classi Presenti : "+str(benchmark.n_classes))
model = SimpleCNN_icarl(num_classes=benchmark.n_classes)

print(f"Training model: {model.__class__.__name__}")


Loading labels...
Loading LUP...
Loading labels names...
Files already downloaded and verified
Classi Presenti : 10
Training model: SimpleCNN_icarl


In [9]:

# Plugin per la valutazione
eval_plugin = EvaluationPlugin(    
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),\
    class_accuracy_metrics(stream=True),\
    loggers=[tb_logger_run1])

# Imposta l'ottimizzatore
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
criterion = nn.CrossEntropyLoss()


In [10]:
model

SimpleCNN_icarl(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=8192, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [5]:

# Configurare la strategia iCarl
icarl = ICaRL(
    model.features,model.classifier, optimizer,
    memory_size=2000, buffer_transform=transform,
    fixed_memory=True, train_mb_size=32, train_epochs=10, eval_mb_size=100,
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    evaluator=eval_plugin
)

for experience in benchmark.train_stream:
    print(f"Training on experience {experience.current_experience}")
    icarl.train(experience)
    print("Training completed")

    print("Evaluating on test stream")
    icarl.eval(benchmark.test_stream)
    print("Evaluation completed")


c:\Users\tellg\Documents\Universita\VISIOPE\2023-2024\Progetto\ContinualLearning\.venv\lib\site-packages\avalanche\training\templates\base.py:468: PositionalArgumentsDeprecatedWarning: Avalanche is transitioning to strategy constructors that accept named (keyword) arguments only. This is done to ensure that there is no confusion regarding the meaning of each argument (strategies can have many arguments). Your are passing 3 positional arguments to the ICaRL.__init__ method. Consider passing them as names arguments. The ability to pass positional arguments will be removed in the future.
  warnings.warn(error_str, category=PositionalArgumentsDeprecatedWarning)


Training on experience 0


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32768x16 and 8192x512)